# Practicing Linear Estimation and Control

Let's explore a estimation and control of a simple, linear, double integrator system

In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from copy import deepcopy

_HEIGHT = -1.0
_VEL_CONST = 1.0
_TIME_STEP = 0.05
_THETA_LIMIT = np.pi/4.0

## Plant Dynamics, Sensors, and Actuators

The following object contains functions for the plant dynamics, sensing of position and velocity, and actuator for acceleration command $a_{\text{cmd}}$

The system we are handling is a simple double integrator in one dimension where the acceleration is the control input. The equations of motion for this system are as:

\begin{equation*}
\dot{\mathbf{x}}  =  \mathbf{f}(\mathbf{x}(t), \mathbf{u}(t)) + \mathbf{w}(t) = \mathbf{F}\mathbf{x}(t) + \mathbf{B}\mathbf{u}(t) + \mathbf{w}(t)
\end{equation*}

where

\begin{equation*}
\mathbf{x}  =  \begin{bmatrix}
x \\
v
\end{bmatrix}, \quad  \mathbf{u} = a_{\text{cmd}}, \quad \mathbf{F}  =  \begin{bmatrix}
1 & dt \\
0 &  1
\end{bmatrix}, \quad \mathbf{B}  =  \begin{bmatrix}
\frac{1}{2}dt^{2} \\
dt
\end{bmatrix}, \quad \mathbf{w} \sim \mathcal{N}(\mathbf{0}, \mathbf{Q}(t))
\end{equation*}

The true, actual, or 'actuated' control input (i.e. acceleration) is assumed/modeled to be commanded acceleration plus zero-mean gaussian noise:

\begin{equation*}
a = a_{\text{cmd}} + a_{\text{noise}} = a_{\text{cmd}} + \mathcal{N}(0, \sigma^2_{a, \text{act}})
\end{equation*}



The sensor readings for position and velocity are assumed/modeled to be the true position and velocity plus zero-mean gaussian noise:

\begin{equation*}
\mathbf{z} = \mathbf{H} \mathbf{x}(t) + \mathbf{v}(t)
\end{equation*}

\begin{equation*}
z_x = x + \mathcal{N}(0, \sigma^2_{x, \text{sens}})
\end{equation*}

\begin{equation*}
z_v = v + \mathcal{N}(0, \sigma^2_{v, \text{sens}})
\end{equation*}

In [ ]:
def eom_double_integrator_1d(s, t, a):
    x, v = s
    dsdt = [v, a]
    return dsdt

class DoubleIntegrator1D():
    '''Object that defines the dynamics of the simple slide-camera'''
    
    def __init__(self, x_0, v_0, a_0, x_T):
        
        # initialize state and control variables
        self.__x = x_0
        self.__v = v_0
        self.__a = a_0
        self.a_cmd = a_0
        
        # target position (hidden)
        self.x_T = x_T
        
        # sensor/actuator noise
        self.z_k = None
        self.__noise_std = dict()
        self.__noise_std['x_s'] = 1.0
        self.__noise_std['v_s'] = 0.5
        self.__noise_std['a_s'] = 0.0
        self.__noise_std['x_a'] = 0.0
        self.__noise_std['v_a'] = 0.0
        self.__noise_std['a_a'] = 0.1
        
        
        # data log
        data = dict()
        data['t'] = []
        data['a_cmd'] = []
        data['a_act'] = []
        data['v_act'] = []
        data['x_act'] = []
        self.data = data 
        
    
    def __get_position(self):
        '''return true position of quadrotor'''
        return self.__x
    
    def get_target_position(self):
        '''return true target position'''
        return self.x_T
    
    def __get_velocity(self):
        '''return true velocity'''
        return self.__v
    
    def __get_acceleration(self):
        '''return true acceleration'''
        return self.__a
    
    def sense_position(self):
        '''sense velocity [m/s] (includes noise)'''
        if self.__noise_std['x_s'] > 0:
            return self.__x + np.random.normal(0.0, self.__noise_std['x_s'])
        else:
            return self.__x
    
    def sense_velocity(self):
        '''sense velocity [m/s] (includes noise)'''
        if self.__noise_std['v_s'] > 0:
            return self.__v + np.random.normal(0.0, self.__noise_std['v_s'])
        else:
            return self.__v
    
    def sense_acceleration(self):
        '''sense acceleration [m/s^2] (includes noise)'''
        if self.__noise_std['a_s'] > 0:
            return self.__a + np.random.normal(0.0, self.__noise_std['a_s'])
        else:
            return self.__a
        

    def actuate_acceleration_command(self, a_cmd, dt):
        '''apply actuation to system and propagate dynamics'''
        
        # apply actuation noise
        self.a_cmd = a_cmd
        if self.__noise_std['a_a'] > 0:
            a = a_cmd + np.random.normal(0.0, self.__noise_std['a_a'])
        else:
            a = a_cmd
        
        # integrate differential equations
        x = deepcopy(self.__x)
        v = deepcopy(self.__v)
        s = odeint(eom_double_integrator_1d, (x, v), np.array([0.0, dt]), args=(a,))
        
        # update state and control variables
        self.__a = a
        self.__v = s[-1,1]
        self.__x = s[-1,0] 
        
    def log_data(self, t):
        '''log truth (act) and sesnor (sens) data '''
        
        self.data['t'].append(t)
        self.data['a_cmd'].append(self.a_cmd)
        self.data['a_act'].append(self.__a)
        self.data['v_act'].append(self.__v)
        self.data['x_act'].append(self.__x)

## Controllers

Functions for control algorithms such as proportional control, proportional-derivative control, etc, as well as any custom controllers you may wish to try

In [ ]:
def p_control(y_err, kp):
    ''' compute the actuator command based on proportional error between output and desired output
    Args:
    y_err: y_des - y where y is the output variable of the plant
    '''
    
    # TODO: write a proportional control law (hint: it is a single line, very simple equations)
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return cmd

In [ ]:
def pd_control(y_err, y_err_prev, dt, kp, kd):
    '''compute the actuator command based on proportional and derivative error between output and target
    Args:
    y_err: y_des - y where y is the output variable of the plant
    y_err_prev: previous stepp y_err
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return cmd

In [ ]:
def pd2_control(y_err0, y_err1, y_err2, dt, kp, kd2):
    '''compute the actuator command based on proportional and double derivative error between output and target
    Args:
    y_err0: y_des - y where y is the output variable of the plant
    y_err1: previous y_err0
    y_err2: previous y_err1
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return cmd

## Estimators

State estimation algorithms. See references:
    - https://en.wikipedia.org/wiki/Kalman_filter
    - https://en.wikipedia.org/wiki/Extended_Kalman_filter
    - http://web.mit.edu/kirtley/kirtley/binlustuff/literature/control/Kalman%20filter.pdf
    - https://home.wlu.edu/~levys/kalman_tutorial/
    
In order to estimate the state of the system (i.e. position and velocity), we will use an Extended Kalman Filter (EKF). Note that, since this system is linear, the Extended Kalman Filter reduces identically to a Kalman Filter. In this tutorial we muddle the line between these two, presenting the mathematics of the Kalman Filter while maintaining the nomenclature of 'EKF'. The distinction between the two will be made more explicit in later tutorials that apply to nonlinear systems.

Here we will use the discrete-time formulation of the EKF. This filter involves a prediction step that propagates the current state estimate and control command a small step into the future, as well as an update step that corrects the prediction based on the measurement. The equations below introduce the $\mathtt{predict}$ and $\mathtt{update}$ algorithms along with the nomenclature to be used

__predict:__

\begin{equation*}
\hat{\mathbf{x}}_{k|k-1} = \mathbf{F}_k \hat{\mathbf{x}}_{k-1|k-1} + \mathbf{B}_k \mathbf{u}_k
\end{equation*}

\begin{equation*}
\hat{\mathbf{P}}_{k|k-1} = \mathbf{F}_k \hat{\mathbf{P}}_{k-1|k-1} \mathbf{F}^{\text{T}}_k+ \mathbf{Q}_k
\end{equation*}

__update:__

\begin{align*}
& \tilde{\mathbf{y}}_k = \mathbf{z}_k - \mathbf{H}_k \hat{\mathbf{x}}_{k|k-1} \\
& \mathbf{S}_k = \mathbf{R}_k + \mathbf{H}_k \mathbf{P}_{k|k-1} \mathbf{H}^{\text{T}}_k \\
& \mathbf{K}_k = \mathbf{P}_{k|k-1} \mathbf{H}^{\text{T}}_k \mathbf{S}^{-1} \\
& \hat{\mathbf{x}}_{k|k} = \hat{\mathbf{x}}_{k|k-1} + \mathbf{K}_k \tilde{\mathbf{y}}_k \\
& \mathbf{P}_{k|k} = \left( \mathbf{I} - \mathbf{K}_k \mathbf{H}_k \right) \mathbf{P}_{k|k-1}
\end{align*}

Note that the only things to be input by us the user are the initial conditions $\hat{\mathbf{x}}_{0|0}$ and $\mathbf{P}_{0|0}$, the process covariance matrix $\mathbf{Q}_k$, and the sensor covariance matrix $\mathbf{R}_k$.

__initial conditions:__

If we assume we have perfect knowledge of the initial state being $[x_0, v_0]^{\text{T}}$ we can initialize as follows:

\begin{equation*}
\hat{\mathbf{x}}_{0|0} = \begin{bmatrix}
x_0 \\ v_0
\end{bmatrix}, \quad \mathbf{P}_{0|0} = \begin{bmatrix}
0 & 0 \\
0 & 0
\end{bmatrix}
\end{equation*}

__covariance matrices:__

It is common to see the process and sensor noise covariance matrices assumed as diagonal, i.e. process variables are independent of one another and sensor variables are independent of one another. Since we have not explicitly stated how we are measuring position and velocity, it may be reasonable to assume that these measurements are independent, that is to say $\mathbf{R}_k$ is a diagonal matrix of form

\begin{equation*}
\mathbf{R}_k = \begin{bmatrix}
\hat{\sigma}^2_{x,\text{sens}} & 0 \\
0 & \hat{\sigma}^2_{v,\text{sens}}
\end{bmatrix}
\end{equation*}

We should be careful here not to make the mistake of assuming that the process noise covariance matrix is of the same form. We in fact have a model of the process dynamics and we should use this to derive the form of the matrix $\mathbf{Q}_k$. First note that we model the effect of the process noise as

\begin{equation*}
\mathbf{w}_k = \mathbf{B}_k a_{\text{noise}} \rightarrow \mathbf{w}_k \sim \mathbf{B} \mathcal{N}(0, \hat{\sigma}_{a,\text{sens}})
\end{equation*}

and use this to derive 

\begin{align*}
\mathbf{Q}_k & = \text{Cov}(\mathbf{w}_k) \\
& = \text{E} \left[ (\mathbf{w}_k - \text{E}[\mathbf{w}_k]) (\mathbf{w}_k - \text{E}[\mathbf{w}_k])^{\text{T}} \right] \\
& = \text{E} \left[ (\mathbf{w}_k - 0) (\mathbf{w}_k - 0)^{\text{T}} \right] \\
& = \text{E} \left[ \mathbf{w}_k \mathbf{w}_k^{\text{T}} \right] \\
& = \text{E} \left[ \mathbf{B}_k a_{\text{noise}} a_{\text{noise}} \mathbf{B}_k^{\text{T}} \right] \\
& = \mathbf{B}_k \mathbf{B}_k^{\text{T}} \text{E} \left[ a^2_{\text{noise}} \right] \\
\end{align*}

Now note, by definition of variance:

\begin{align*}
& \quad \text{Var}(a_{\text{noise}}) = \sigma^2_{a,\text{act}} \\
& \quad \text{Var}(a_{\text{noise}}) = \text{E}[a^2_{\text{noise}}] - \text{E}[a_{\text{noise}}]^2 = \text{E}[a^2_{\text{noise}}] \\
\therefore & \quad \text{E}[a^2_{\text{noise}}] = \sigma^2_{a,\text{act}}
\end{align*}

Plugging this back into previous equations we get

\begin{align*}
\mathbf{Q}_k & = \mathbf{B}_k \mathbf{B}_k^{\text{T}} \text{E} \left[ a^2_{\text{noise}} \right] \\
& = \mathbf{B}_k \mathbf{B}_k^{\text{T}} \sigma^2_{a,\text{act}} \\
& = \begin{bmatrix} \frac{1}{4} dt^4 & \frac{1}{2} dt^3 \\ \frac{1}{2} dt^3 & dt^2\end{bmatrix} \sigma^2_{a,\text{act}}
\end{align*}

__Now let's apply these equations in code:__

In [ ]:
class EKF_DoubleIntegrator1D():
    '''Extended Kalman Filter for estimating relative position (p), velocity (v) and pitch (theta) of 
            1-D kinematic quadrotor system
     Notation (see https://en.wikipedia.org/wiki/Extended_Kalman_filter):
    - x[0] = $p = x_{T} - x$: position of target relative to position of quadrotor
    - x[1] = $v$: velocity of quadrotor in x direction
    - x[2] = $\theta$: pitch of quadrotor
    - u[0] = $\dot{\theta}$: pitch rate of quadrotor (commanded or measured....?)
    - z[0] = $\gamma$: measured view angle from camera centerline to target
    - z[1] = $a$: measured linear acceleration of quadrotor
    - z[2] = $\theta$: measure pitch of quadrotor
    - C_vtheta = constant that maps pitch to linear acceleration: $\dot{v} = -C_{v\theta} \text{sin}(\theta)$
    - h_z = $z_{T} - z$ height of target relative to height of quadrotor (assumed constant and known)
    - x_k1_k1 = \hat{x}_{k-1|k-1} estimate of state at step k-1 based on step k-1
    - x_k_k1 = \hat{x}_{k|k-1} predicted state at step k based on step k-1
    - F_k = F_{k} jacobian of dynamics f with respect to state x, evaluated at x_{k|k-1}, u_{k}
    - P_k_k1 = P_{k|k-1} predicted covariance at step k based on step k-1
    - P_k1_k1 = P_{k|k-1} estimated covariance at step k-1 based on step k-1
    '''
    
    def __init__(self, x_0, v_0):
        
        # state predictions and estimates
        self.x_k_k1 = np.array(2*[0.0])
        self.x_k_k = np.array([x_0, v_0])
        self.x_k1_k1 = np.array([x_0, v_0])
        
        # actuator (process) covariance
        self.s2_a_a = 0.5
        
        # observation (sensor) covariance
        self.s2_x_s = 1.0
        self.s2_v_s = 0.1
        self.R_k = np.diag([self.s2_x_s, self.s2_v_s])
        
        # covariance predictions and estimates
        self.P_k_k1 = 0.0*np.eye(2)
        self.P_k_k = 0.0*np.eye(2)
        self.P_k1_k1 = 0.0*np.eye(2)
        
        # data log
        self.data = dict()
        self.data['t'] = []
        self.data['x_sens'] = []
        self.data['x_pred'] = []
        self.data['x_est'] = []
        self.data['y_x'] = []
        self.data['v_sens'] = []
        self.data['v_pred'] = []
        self.data['v_est'] = []
        self.data['y_v'] = []
        self.data['K_00'] = []
        self.data['K_01'] = []
        self.data['K_10'] = []
        self.data['K_11'] = []
    
    def predict(self, u_k, dt):
        ''' prediction step of k based on step k-1
        '''
        
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def update(self, z_k):
        ''' estimation update for step k based on step k
        Args:
        - z_k: measurements taken at step k
        '''

        # YOUR CODE HERE
        raise NotImplementedError()
    
    def log_data(self, t):
        '''log estimator data'''
        
        self.data['t'].append(t)
        self.data['x_sens'].append(self.z_k[0])
        self.data['x_pred'].append(self.x_k_k1[0])
        self.data['x_est'].append(self.x_k_k[0])
        self.data['y_x'].append(self.y_k[0])
        self.data['v_pred'].append(self.x_k_k1[1])
        self.data['v_est'].append(self.x_k_k[1])
        if len(self.z_k) > 1:
            self.data['v_sens'].append(self.z_k[1])
            self.data['y_v'].append(self.y_k[1])
            self.data['K_00'].append(self.K_k[0][0])
            self.data['K_01'].append(self.K_k[0][1])
            self.data['K_10'].append(self.K_k[1][0])
            self.data['K_11'].append(self.K_k[1][1])
        else:
            self.data['K_00'].append(self.K_k[0])
            self.data['K_01'].append(self.K_k[1])
    

## Simulation Script

below is a script for testing various controllers for the `SimpleSlideCamera` plant as well as plotting the results.

In [ ]:
# Control inputs
kp = 0.2
kd = 0.01
dt = 0.1
t_final = 20.0

# intial conditions (position, velocity and targe position)
x_0 = 0.0
v_0 = 0.0
a_0 = 0.0
x_T = 1.0

# create SimpleSlideCamera with initial conditions
dblint = DoubleIntegrator1D(x_0, v_0, a_0, x_T)

# initialize EKF state estimator
ekf = EKF_DoubleIntegrator1D(x_0, v_0)

# initialize data storage
t = 0.0
z_k = np.array([dblint.sense_position(), dblint.sense_velocity()])
a_cmd = 0.0
while t < t_final:

    # ESTIMATOR: predict next state
    ekf.predict([a_cmd], dt)
    
    # ESTIMATOR: update state estimate based on sensor data
    z_k = np.array([dblint.sense_position(), dblint.sense_velocity()])
    ekf.update(z_k)
    
    # store data
    dblint.log_data(t)
    ekf.log_data(t)
    
    # CONTROLLER: call theta control algoritm
    err_x = dblint.x_T - z_k[0]
    a_cmd = 1.0*np.sin(t)
    
    # ACTUATOR: send velocity command to plant
    dblint.actuate_acceleration_command(a_cmd, dt)
    
    # increment time step
    t += dt
    

## Analysis

visualize the results of you simulation to analyze performance of estimator and controller

In [ ]:
# plot size
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
# position
h_x_act, = plt.plot(dblint.data['t'], dblint.data['x_act'], label='truth')
h_x_est, = plt.plot(ekf.data['t'], ekf.data['x_est'], label='estimate')
h_x_pred, = plt.plot(ekf.data['t'], ekf.data['x_pred'], label='prediction')
h_x_sens, = plt.plot(ekf.data['t'], ekf.data['x_sens'], label='measurement',linestyle="",marker=".")
plt.legend(handles=[h_x_act, h_x_est, h_x_pred, h_x_sens,])
plt.title('Position', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('x-pos [m]')
plt.show()

In [ ]:
h_v_act, = plt.plot(dblint.data['t'], dblint.data['v_act'], label='truth')
h_v_est, = plt.plot(ekf.data['t'], ekf.data['v_est'], label='estimate')
h_v_pred, = plt.plot(ekf.data['t'], ekf.data['v_pred'], label='prediction')
h_v_sens, = plt.plot(ekf.data['t'], ekf.data['v_sens'], label='measurement',linestyle="",marker=".")
plt.legend(handles=[h_v_act, h_v_sens, h_v_pred, h_v_est])
plt.title('Velocity', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('x-vel [m/s]')
plt.show()

In [ ]:
# acceleration
h_a_act, = plt.plot(dblint.data['t'], dblint.data['a_act'], label='truth')
h_a_cmd, = plt.plot(dblint.data['t'], dblint.data['a_cmd'], label='command')
plt.legend(handles=[h_a_act, h_a_cmd])
plt.title('Acceleration', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('x-acc [m/s/s]')
plt.show()

In [ ]:
# residual
h_y_x, = plt.plot(ekf.data['t'], ekf.data['y_x'], label='position [m]')
h_y_v, = plt.plot(ekf.data['t'], ekf.data['y_v'], label='velocity [m/s]')
plt.legend(handles=[h_y_x, h_y_v])
plt.title('Measurment Residual', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('residual')
plt.show()

In [ ]:
# kalman gain
h_K_00, = plt.plot(ekf.data['t'], ekf.data['K_00'], label='K_00 [m]')
h_K_01, = plt.plot(ekf.data['t'], ekf.data['K_01'], label='K_01 [m]')
h_K_10, = plt.plot(ekf.data['t'], ekf.data['K_10'], label='K_10 [m]')
h_K_11, = plt.plot(ekf.data['t'], ekf.data['K_11'], label='K_11 [m]')
plt.legend(handles=[h_K_00, h_K_01, h_K_10, h_K_11])
plt.title('Kalman Gain', fontsize=20)
plt.xlabel('time [s]')
plt.ylabel('gain')
plt.show()

## Questions

1. Can you design a controller that is capable of converging the position error to zero?
2. We derived the EKF's process noise covariance matrix, $\mathbf{Q}_k$, to incorporate the off-diagonal terms that represent dependence between position and velocity . What happens if you assume the EKF's process noise for position and velocity are independent, i.e. only diagnonal terms in $\mathbf{Q}_k$. Does this make the Kalman Filter less accurate?
3. Since we wrote both the truth model and the kalman filter, it's easy for us to choose the correct process and sensor noises for the kalman filter to match those used in the truth model. Perhaps it is obvious, but it's important to point out that this is almost certainly NOT possible to do in the real world. Coming up with appropriate $\mathbf{Q}_k$ and $\mathbf{R}_k$ matrices is one of the fundamental challenges to implementing a real-world kalman filter. Let's play this a bit. Leaving the truth model's noise variance as is, what is the effect of increasing the EKF's process noise $\mathbf{Q}_k$ above that of the truth model? What about it's sensor noise, $\mathbf{R}_k$? What about decreasing these below the truth model's? 
4. All of the process and signal noise was assumed to be zero-mean gaussian, i.e. no bias. What happens if you change the truth model of the noise (i.e. DoubleIntegrator1D) to have a non-zero bias? Can the EKF still accurately track the truth?
5.  